In [5]:
# Install necessary libraries in Colab
!pip install streamlit pyngrok transformers torch -q

# Import required libraries
import streamlit as st
from pyngrok import ngrok
from transformers import pipeline
import time
import re

# Write the full Streamlit app code to a file
with open("app.py", "w") as f:
    f.write('''import streamlit as st
from transformers import pipeline
import time
import re

# Load pre-trained model for question generation
generator = pipeline("text-generation", model="gpt2")  # Using gpt2 for better results

# Function to generate technical questions using the LLM
def generate_technical_questions(tech_stack):
    """Generate 3-5 technical questions based on the user's tech stack using the LLM."""
    prompt = (
        f"Generate a list of 3 to 5 technical interview questions for a candidate proficient in {tech_stack}. "
        "Focus on practical knowledge and problem-solving skills related to these technologies. "
        "Format the output as a numbered list (e.g., 1. Question text). Provide only the questions, no extra text."
    )
    response = generator(prompt, max_length=300, num_return_sequences=1, temperature=0.8, top_k=50)[0]["generated_text"]

    # Extract questions from the response
    lines = response.split("\\n")
    questions = []
    for line in lines[1:]:  # Skip the prompt
        line = line.strip()
        if line and line[0].isdigit() and "." in line:  # Check for numbered format (e.g., "1. ")
            questions.append(line)

    # Fallback if no questions are generated
    if not questions:
        questions = [
            f"1. Tell me about a project you built using {tech_stack}.",
            f"2. What challenges have you faced with {tech_stack} and how did you overcome them?",
            f"3. Explain a key feature of {tech_stack} that you find most useful."
        ]

    return questions[:5]  # Limit to 5 questions

# Validation functions
def is_valid_email(email):
    """Check if the email format is valid."""
    return re.match(r"[^@]+@[^@]+\\.[^@]+", email) is not None

def is_valid_phone(phone):
    """Check if the phone number is in a basic valid format (e.g., 123-456-7890 or 1234567890)."""
    return re.match(r"\\d{3}[-.]?\\d{3}[-.]?\\d{4}", phone) is not None

def is_valid_experience(experience):
    """Check if experience is a positive number."""
    try:
        return float(experience) >= 0
    except ValueError:
        return False

# Streamlit app logic
def main():
    # Initialize session state for conversation history and candidate info
    if "messages" not in st.session_state:
        st.session_state.messages = []
    if "candidate_info" not in st.session_state:
        st.session_state.candidate_info = {}
    if "step" not in st.session_state:
        st.session_state.step = "greeting"
    if "questions" not in st.session_state:
        st.session_state.questions = []
    if "current_question_index" not in st.session_state:
        st.session_state.current_question_index = 0

    # UI Header
    st.title("TalentScout Hiring Assistant")
    st.write("Welcome! I'm here to assist you with your application for tech positions.")

    # Display conversation history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Stop further input if conversation has ended
    if st.session_state.step == "end":
        return

    # Greeting step
    if st.session_state.step == "greeting":
        greeting = "Hello! I'm the TalentScout Hiring Assistant. My purpose is to help you apply for tech positions by gathering your information and assessing your skills. Let’s get started! Please provide your full name."
        st.session_state.messages.append({"role": "assistant", "content": greeting})
        st.session_state.step = "name"
        st.rerun()

    # User input
    user_input = st.chat_input("Type your response here...")

    if user_input:
        # Add user message to history
        st.session_state.messages.append({"role": "user", "content": user_input})

        # Handle conversation flow based on step
        with st.chat_message("user"):
            st.markdown(user_input)

        with st.chat_message("assistant"):
            if st.session_state.step == "name":
                if user_input.strip():
                    st.session_state.candidate_info["name"] = user_input.strip()
                    response = "Nice to meet you, {}! Please provide your email address.".format(user_input)
                    st.session_state.step = "email"
                else:
                    response = "Please provide a valid full name to continue."
            elif st.session_state.step == "email":
                if is_valid_email(user_input):
                    st.session_state.candidate_info["email"] = user_input
                    response = "Thanks! Now, please provide your phone number (e.g., 123-456-7890)."
                    st.session_state.step = "phone"
                else:
                    response = "That doesn’t look like a valid email address. Please try again (e.g., example@domain.com)."
            elif st.session_state.step == "phone":
                if is_valid_phone(user_input):
                    st.session_state.candidate_info["phone"] = user_input
                    response = "Great! How many years of experience do you have?"
                    st.session_state.step = "experience"
                else:
                    response = "Please provide a valid phone number (e.g., 123-456-7890 or 1234567890)."
            elif st.session_state.step == "experience":
                if is_valid_experience(user_input):
                    st.session_state.candidate_info["experience"] = user_input
                    response = "Got it! What position(s) are you applying for?"
                    st.session_state.step = "position"
                else:
                    response = "Please provide a valid number of years (e.g., 3 or 5.5)."
            elif st.session_state.step == "position":
                if user_input.strip():
                    st.session_state.candidate_info["position"] = user_input.strip()
                    response = "Thanks! Where are you currently located?"
                    st.session_state.step = "location"
                else:
                    response = "Please specify at least one position you’re applying for."
            elif st.session_state.step == "location":
                if user_input.strip():
                    st.session_state.candidate_info["location"] = user_input.strip()
                    response = "Perfect! Please tell me about your tech stack (e.g., programming languages, frameworks, tools)."
                    st.session_state.step = "tech_stack"
                else:
                    response = "Please provide your current location."
            elif st.session_state.step == "tech_stack":
                if user_input.strip():
                    st.session_state.candidate_info["tech_stack"] = user_input.strip()
                    st.session_state.questions = generate_technical_questions(user_input)
                    st.session_state.current_question_index = 0
                    response = f"Here’s your first technical question:\\n{st.session_state.questions[0]}\\nPlease provide your answer."
                    st.session_state.step = "questions"
                else:
                    response = "Please specify your tech stack (e.g., Python, Flask)."
            elif st.session_state.step == "questions":
                if "goodbye" in user_input.lower() or "exit" in user_input.lower():
                    response = "Thank you for chatting with me, {}! A recruiter will review your information and get back to you soon. Goodbye!".format(st.session_state.candidate_info["name"])
                    st.session_state.step = "end"
                else:
                    # Check if the answer is meaningful
                    if len(user_input.strip()) < 5 or user_input.lower() in ["idk", "i don’t know", "skip"]:
                        response = f"I didn’t quite get that. Could you provide more detail for this question?\\n{st.session_state.questions[st.session_state.current_question_index]}\\nPlease try again."
                    else:
                        # Move to the next question if available
                        st.session_state.current_question_index += 1
                        if st.session_state.current_question_index < len(st.session_state.questions):
                            response = f"Thanks for your answer! Here’s the next question:\\n{st.session_state.questions[st.session_state.current_question_index]}\\nPlease provide your answer."
                        else:
                            response = "Thanks for answering all the questions! You can say 'goodbye' or 'exit' to end the conversation, or ask me anything else."
            else:
                response = "I'm here to assist you with your application. How can I help you next?"

            # Add assistant response to history
            st.session_state.messages.append({"role": "assistant", "content": response})
            st.markdown(response)

            # Rerun to reflect the "end" state immediately
            if st.session_state.step == "end":
                st.rerun()

if __name__ == "__main__":
    main()
''')

# Set up ngrok and run Streamlit
!ngrok authtoken 2uF5zuwLTgYkh567oQ1hAmUyx1U_7kboSYmKYQg6JAhAXvxFz  # Your provided ngrok auth token
public_url = ngrok.connect(8501)
print(f"Access the app at: {public_url}")

# Run Streamlit app
!streamlit run app.py --server.port 8501

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Access the app at: NgrokTunnel: "https://206c-35-223-210-115.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.223.210.115:8501

2025-03-17 06:27:49.761608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742192869.780772   16369 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742192869.786186   16369 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cpu
2025-03-17 06:27:56.791 Examining the path of torch.classes raise